In [10]:
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup
import nest_asyncio

import pytesseract
from PIL import Image
import pyautogui
import cv2
import time
import numpy as np
import subprocess
import webbrowser

import sys, os

# Go one level up from the notebook folder
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from img_to_text.ocr import extract_text_strong

# ===== CONFIG =====
URL = "https://portfolio-praveen-gupta.vercel.app/"
URL = "https://practice.expandtesting.com/contact?utm_source=chatgpt.com"
SCROLL_AMOUNT = -50
SCROLL_DELAY = 0.2
SCROLL_LIMIT = 200

# --- Screenshot helper ---
def take_screenshot():
    screenshot_path = "/home/praveen/Desktop/My-Projects/interview_p/imgs/outputs/screen.png"
    time.sleep(0.5)
    subprocess.run(["gnome-screenshot", "-f", screenshot_path])
    return screenshot_path

# --- OCR extraction ---
def extract_text(image_path):
    return extract_text_strong(image_path)  # use your function directly

# --- Find coordinates of text in image ---
def find_text_coordinate_in_image(image_path, search_text):
    img = Image.open(image_path).convert("L")
    img = img.point(lambda x: 0 if x < 130 else 255)

    boxes = pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)
    for i in range(len(boxes['text'])):
        if search_text.lower() in boxes['text'][i].lower():
            x, y, w, h = boxes['left'][i], boxes['top'][i] + 25, boxes['width'][i], boxes['height'][i] + 25
            center_x = x + w // 2
            center_y = y + h // 2
            return (center_x, center_y)
    return None

# --- Fill field and press Tab ---
def fill_field(value):
    pyautogui.typewrite(value)
    pyautogui.press("tab")
    time.sleep(0.5)

# Allow nested event loops (needed for Jupyter/IPython)
nest_asyncio.apply()

async def fetch_form():
    session = AsyncHTMLSession()
    webbrowser.open(URL)

    r = await session.get(URL)
    await r.html.arender(timeout=20, sleep=2)

    soup = BeautifulSoup(r.html.html, "html.parser")
    form = soup.find("form")

    if not form:
        print("[ERROR] No form found on page.")
        return []

    fields_bs4 = form.find_all(["input", "textarea"])
    field_names = []
    for tag in fields_bs4:
        if tag.get("type") in ["submit", "hidden", "checkbox", "radio"]:
            continue
        placeholder = tag.get("name") or tag.get("name") or tag.get("id") or tag.text
        if placeholder:
            field_names.append(placeholder.strip())

    print("[INFO] Required fields:", field_names)
    return field_names

async def main():
    field_names = await fetch_form()
    if not field_names:
        return

    # Prepare input data safely
    input_data = {}
    if len(field_names) >= 3:
        input_data = {
            field_names[0]: "John Doe",
            field_names[1]: "john@example.com",
            field_names[2]: "This is a test message"
        }
    else:
        for name in field_names:
            input_data[name] = f"Sample for {name}"

    # Locate first field by OCR
    target_label = field_names[0]
    coord = None
    for attempt in range(SCROLL_LIMIT):
        shot = take_screenshot()
        coord = find_text_coordinate_in_image(shot, target_label)
        if coord:
            print(f"[FOUND] '{target_label}' at {coord} (attempt {attempt+1})")
            pyautogui.click(coord[0], coord[1])
            fill_field(input_data[target_label])
            input_data[target_label] = ""  # mark filled
            break
        print(f"[INFO] '{target_label}' not found, scrolling...")
        pyautogui.scroll(SCROLL_AMOUNT)
        time.sleep(SCROLL_DELAY)

    # Fill remaining fields sequentially via Tab
    for fname, fvalue in input_data.items():
        if fvalue:
            fill_field(fvalue)

    # Press Enter to submit
    pyautogui.press('enter')

# Run the main function
await main()


[ERROR] No form found on page.


In [ ]:
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup
import nest_asyncio

import pytesseract
from PIL import Image
import pyautogui
import cv2
import time
import numpy as np
import subprocess
import webbrowser

import sys, os

# Go one level up from the notebook folder
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from img_to_text.ocr import extract_text_strong

# ===== CONFIG =====
URL = "https://portfolio-praveen-gupta.vercel.app/"
URL = "https://practice.expandtesting.com/contact?utm_source=chatgpt.com"
SCROLL_AMOUNT = -50
SCROLL_DELAY = 0.2
SCROLL_LIMIT = 200

# --- Screenshot helper ---
def take_screenshot():
    screenshot_path = "/home/praveen/Desktop/My-Projects/interview_p/imgs/outputs/screen.png"
    time.sleep(0.1)
    subprocess.run(["gnome-screenshot", "-f", screenshot_path])
    return screenshot_path

# --- OCR extraction ---
def extract_text(image_path):
    return extract_text_strong(image_path)  # use your function directly

# --- Find coordinates of text in image ---
def find_text_coordinate_in_image(image_path, search_text):
    img = Image.open(image_path).convert("L")
    img = img.point(lambda x: 0 if x < 130 else 255)

    boxes = pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)
    for i in range(len(boxes['text'])):
        if search_text.lower() in boxes['text'][i].lower():
            x, y, w, h = boxes['left'][i], boxes['top'][i] + 25, boxes['width'][i], boxes['height'][i] + 25
            center_x = x + w // 2
            center_y = y + h // 2
            return (center_x, center_y)
    return None

# --- Fill field and press Tab ---
def fill_field(value):
    pyautogui.typewrite(value)
    pyautogui.press("tab")
    time.sleep(0.5)

# Allow nested event loops (needed for Jupyter/IPython)
nest_asyncio.apply()

async def fetch_form():
    session = AsyncHTMLSession()
    webbrowser.open(URL)

    r = await session.get(URL)
    await r.html.arender(timeout=20, sleep=1)

    soup = BeautifulSoup(r.html.html, "html.parser")
    

    fields_bs4 = soup.find_all(class_='form-label')
    field_names = []
    for tag in fields_bs4:
        if tag.get("type") in ["submit", "hidden", "checkbox", "radio"]:
            continue
        placeholder = tag.text.strip()
        if placeholder:
            field_names.append(placeholder.strip())

    print("[INFO] Required fields:", field_names)
    return field_names

async def main():
    field_names = await fetch_form()
    if not field_names:
        return

    # Prepare input data safely
    input_data = {}
    if len(field_names) >= 3:
        input_data = {
            field_names[0]: "John Doe",
            field_names[1]: "john@example.com",
            field_names[2]: "This is a test message"
        }
    else:
        for name in field_names:
            input_data[name] = f"Sample for {name}"

    # Locate first field by OCR
    target_label = field_names[0]
    coord = None
    for attempt in range(SCROLL_LIMIT):
        shot = take_screenshot()
        coord = find_text_coordinate_in_image(shot, target_label)
        if coord:
            print(f"[FOUND] '{target_label}' at {coord} (attempt {attempt+1})")
            pyautogui.click(coord[0], coord[1])
            fill_field(input_data[target_label])
            input_data[target_label] = ""  # mark filled
            break
        print(f"[INFO] '{target_label}' not found, scrolling...")
        pyautogui.scroll(SCROLL_AMOUNT)
        time.sleep(SCROLL_DELAY)

    # Fill remaining fields sequentially via Tab
    for fname, fvalue in input_data.items():
        if fvalue:
            fill_field(fvalue)

    # Press Enter to submit
    pyautogui.press('enter')

# Run the main function
await main()


[INFO] Required fields: ['Name', 'Email', 'You message']
[FOUND] 'Name' at (379, 615) (attempt 1)


# another try

In [ ]:
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup
import nest_asyncio

import pytesseract
from PIL import Image, ImageDraw
import pyautogui
import cv2
import time
import numpy as np
import subprocess
import webbrowser

import sys, os

# Go one level up from the notebook folder
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from img_to_text.ocr import extract_text_strong

# ===== CONFIG =====
URL = "https://portfolio-praveen-gupta.vercel.app/"
URL = "https://practice.expandtesting.com/contact?utm_source=chatgpt.com"
SCROLL_AMOUNT = -50
SCROLL_DELAY = 0.2
SCROLL_LIMIT = 200

# --- Screenshot helper ---
def take_screenshot():
    screenshot_path = "/home/praveen/Desktop/My-Projects/interview_p/imgs/outputs/screen.png"
    time.sleep(0.5)
    subprocess.run(["gnome-screenshot", "-f", screenshot_path])
    return screenshot_path

# --- OCR extraction ---
def extract_text(image_path):
    return extract_text_strong(image_path)  # use your function directly

# --- Find coordinates of text in image ---
def find_input_box_coordinate(image_path, label_text):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Increase contrast
    gray = cv2.equalizeHist(gray)

    # Detect edges
    edges = cv2.Canny(gray, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # First: find label coordinates via OCR
    boxes = pytesseract.image_to_data(gray, output_type=pytesseract.Output.DICT)
    label_coord = None
    for i, text in enumerate(boxes['text']):
        if label_text.lower() in text.lower():
            label_coord = (boxes['left'][i], boxes['top'][i] + boxes['height'][i] // 2)
            break
    if not label_coord:
        return None

    lx, ly = label_coord

    # Second: find the nearest horizontal rectangle below label
    possible_inputs = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = w / float(h)
        if aspect_ratio > 3 and 15 < h < 60:  # looks like a text box
            possible_inputs.append((x, y, w, h))

    # Find the input box just below the label
    possible_inputs.sort(key=lambda b: abs(b[1] - ly))  # nearest vertically
    for (x, y, w, h) in possible_inputs:
        if y > ly:  # below the label
            clr_img = Image.open("/home/praveen/Desktop/My-Projects/interview_p/imgs/outputs/screen.png")
            draw = ImageDraw.Draw(clr_img)
            draw.rectangle([(x, y), (x + w, y + h)], outline="red", width=2)
            clr_img.save('/home/praveen/Desktop/My-Projects/interview_p/imgs/outputs/boxxed_img.png')
            return (x + w // 2, y + h // 2)

    return None

# --- Fill field and press Tab ---
def fill_field(value):
    pyautogui.typewrite(value)
    pyautogui.press("tab")
    time.sleep(0.5)

# Allow nested event loops (needed for Jupyter/IPython)
nest_asyncio.apply()

async def fetch_form():
    session = AsyncHTMLSession()
    webbrowser.open(URL)

    r = await session.get(URL)
    await r.html.arender(timeout=20, sleep=1)

    soup = BeautifulSoup(r.html.html, "html.parser")
    

    fields_bs4 = soup.find_all(class_='form-label')
    field_names = []
    for tag in fields_bs4:
        if tag.get("type") in ["submit", "hidden", "checkbox", "radio"]:
            continue
        placeholder = tag.text.strip()
        if placeholder:
            field_names.append(placeholder.strip())

    print("[INFO] Required fields:", field_names)
    return field_names

async def main():
    field_names = await fetch_form()
    if not field_names:
        return

    # Prepare input data safely
    input_data = {}
    if len(field_names) >= 3:
        input_data = {
            field_names[0]: "John Doe",
            field_names[1]: "john@example.com",
            field_names[2]: "This is a test message"
        }
    else:
        for name in field_names:
            input_data[name] = f"Sample for {name}"

    # Locate first field by OCR
    target_label = field_names[0]
    coord = None
    for attempt in range(SCROLL_LIMIT):
        shot = take_screenshot()
        coord = find_text_coordinate_in_image(shot, target_label)
        if coord:
            print(f"[FOUND] '{target_label}' at {coord} (attempt {attempt+1})")
            pyautogui.click(coord[0], coord[1])
            fill_field(input_data[target_label])
            input_data[target_label] = ""  # mark filled
            break
        print(f"[INFO] '{target_label}' not found, scrolling...")
        pyautogui.scroll(SCROLL_AMOUNT)
        time.sleep(SCROLL_DELAY)

    # Fill remaining fields sequentially via Tab
    for fname, fvalue in input_data.items():
        if fvalue:
            fill_field(fvalue)

    # Press Enter to submit
    pyautogui.press('enter')

# Run the main function
await main()


[INFO] Required fields: ['Name', 'Email', 'You message']
[FOUND] 'Name' at (379, 578) (attempt 1)
